<a href="https://colab.research.google.com/github/CalvHobbes/shop_india/blob/main/llamaindex/price%20comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4 requests

In [2]:
# prompt: check if in colab environment and if so, fetch helper functions for tools

import sys

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py


--2025-01-30 13:49:59--  https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1811 (1.8K) [text/plain]
Saving to: ‘search_Croma.py’

search_Croma.py     100%[===================>]   1.77K  --.-KB/s    in 0s      

2025-01-30 13:49:59 (24.5 MB/s) - ‘search_Croma.py’ saved [1811/1811]

--2025-01-30 13:49:59--  https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1460

In [ ]:
# Import the necessary functions
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance
import json

def test_functions():
  # Define the product query
  query = "iphone 16 pro max"

  # Fetch prices from Croma
  croma_prices = fetch_prices_from_croma(query)
  print("Croma Prices:")
  print(json.dumps(croma_prices, indent=4))

  # Fetch prices from Reliance Digital
  reliance_prices = fetch_prices_from_reliance(query)
  print("Reliance Digital Prices:")
  print(json.dumps(reliance_prices, indent=4))


# test_functions()





In [ ]:
!pip install llama-index llama-index-llms-openai

In [9]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAPI_KEY')

In [12]:

croma_tool = FunctionTool.from_defaults(
    fn=fetch_prices_from_croma,
    name="fetch_prices_from_croma",
    description="Fetches prices for a product from Croma.",
)

reliance_tool = FunctionTool.from_defaults(
    fn=fetch_prices_from_reliance,
    name="fetch_prices_from_reliance",
    description="Fetches prices for a product from Reliance Digital.",
)


In [19]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI

# model = OpenAI(model="gpt-3.5-turbo")
model = OpenAI(model="gpt-4o-mini")



In [20]:
from llama_index.agent.openai import OpenAIAgent

instructions = '''
You are an intelligent assistant tasked with analyzing and combining product price data by executing one or more tools available to you. to run the tools you must provide them the name of the product the user is searching.. Your goal is to identify the most relevant product prices and present them in a structured format.  Execute all applicable tools.Follow these steps:

Combine Results: Analyze the product data from all websites. If product names or display names are slightly different but refer to the same product, use your judgment to group them together.

Identify the Best Prices: For each product, compare the prices across all websites and order them in ascending order (lowest price first).

Output Format: Present the results in the following structured format for each product:

Product Name: The most accurate or common name for the product.

For each store, include:

Store Display Name: The name of the store.

Product URL: A clickable link to the product page.

Price: The price of the product at that store.

Ensure the store-wise details are ordered by ascending price (cheapest first).

Handle Ambiguities: If product names or details are unclear, use context to determine if they refer to the same product. If uncertain, list them separately.

Sample Result:
Product Name: Wireless Noise-Cancelling Headphones
- Store: TechWorld, Product URL: [TechWorld Link], Price: $120
- Store: GadgetHub, Product URL: [GadgetHub Link], Price: $125
- Store: ElectroShop, Product URL: [ElectroShop Link], Price: $130

'''

# Initialize the OpenAIAgent with the tools and model
agent = OpenAIAgent.from_tools(
    [croma_tool, reliance_tool],
    llm=model,
    verbose=True,
    instructions=instructions
)


In [21]:

chat_history = []
response = agent.chat("What is the best price for iphone 16 pro max?", chat_history=chat_history)
print(str(response))

Added user message to memory: What is the best price for iphone 16 pro max?
=== Calling Function ===
Calling function: fetch_prices_from_croma with args: {"product": "iphone 16 pro max"}
Got output: [{'name': 'Apple iPhone 16 Pro Max (256GB, Desert Titanium)', 'url': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746', 'price': 137900.0}, {'name': 'Apple iPhone 16 Pro Max (256GB, Natural Titanium)', 'url': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747', 'price': 137900.0}, {'name': 'Apple iPhone 16 Pro Max (256GB, Black Titanium)', 'url': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742', 'price': 137900.0}, {'name': 'Apple iPhone 16 Pro Max (256GB, White Titanium)', 'url': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744', 'price': 137900.0}, {'name': 'Apple iPhone 16 Pro Max (512GB, Black Titanium)', 'url': 'https://www.croma.com/apple-iphone-16-pro-max-512gb-black-